## Import Libraries and Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
from sklearn import datasets
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.utils.data import Subset
import keras
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Concatenate, Dense, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.regularizers import L2
from keras.metrics import Precision, Recall, AUC
from keras.utils import to_categorical
from keras.optimizers import Adam
import pickle
import warnings
warnings.filterwarnings('ignore')

## Import Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os

# ADC data
data_path = '/content/drive/MyDrive/Computer Vision Project/Data/20230508'
file_names = os.listdir(data_path)
pkl_files = [file for file in file_names if file.endswith('.pkl')]
pkl_files = [pkl_files[0], pkl_files[2], pkl_files[4]]

file_path_y = data_path + '/' + pkl_files[0]
file_path_x = data_path + '/' + pkl_files[1]
file_path_idx = data_path + '/' + pkl_files[2]

with open(file_path_x, 'rb') as file:
  x = pickle.load(file)

with open(file_path_y, 'rb') as file:
  y = pickle.load(file)

with open(file_path_idx, 'rb') as file:
  idx = pickle.load(file)

Mounted at /content/drive


In [ ]:
pkl_files

['20230508_adc_annotated_label.pkl',
 '20230508_adc_input_rgb_128_all.pkl',
 '20230508_patients_index_updated.pkl']

In [ ]:
# Get a list of the keys
keys = list(idx.keys())

# Get the key at the desired index
train_key = keys[np.int(0.7*len(keys))]
print('Last index in training = ', idx[train_key][1])

val_key = keys[np.int(0.85*len(keys))]
print('Last index in validation = ', idx[val_key][1])

Last index in training =  19160
Last index in validation =  23234


## Split Training / Validation / Test

In [ ]:
# Get a list of the keys
keys = list(idx.keys())

# Get the keys at the desired index
train_key = keys[np.int(0.7*len(keys))] # outputs a list [start_idx, end_idx]
val_key = keys[np.int(0.85*len(keys))]
print(train_key, val_key)

# Split into training and validation and test
x_train = x[0:idx[train_key][1]]
y_train = y[0:idx[train_key][1]]

x_val = x[idx[train_key][1]:idx[val_key][1]]
y_val = y[idx[train_key][1]:idx[val_key][1]]

x_test = x[idx[val_key][1]:]
y_test = y[idx[val_key][1]:]

# Print shapes
print('x_train shape = ', x_train.shape)
print('y_train shape = ', y_train.shape)

print('x_val shape = ', x_val.shape)
print('y_val shape = ', y_val.shape)

print('x_test shape = ', x_test.shape)
print('y_test shape = ', y_test.shape)

11035_1001055 11256_1001279
x_train shape =  (19160, 128, 128, 3)
y_train shape =  (19160,)
x_val shape =  (4074, 128, 128, 3)
y_val shape =  (4074,)
x_test shape =  (4108, 128, 128, 3)
y_test shape =  (4108,)


In [ ]:
print(val_key)
print(idx[val_key])

11256_1001279
[23215, 23234]


## Redefine Class Weights

In [ ]:
import numpy as np
from sklearn.utils import class_weight
from collections import Counter

# Assuming you have your labels as a NumPy array or list
labels = y_train

# Find the sample sizes of each class
total_samples = x_train.shape[0]

def sample_per_class_counter(data):
  samples_per_class = []
  total_samples = data.shape[0]

  for class_label in np.unique(data):
    counter = 0
    for i in data:
      if i == class_label:
        counter += 1
    
    samples_per_class.append(counter)
  
  return samples_per_class

# Calculate class weights
sample_weights = class_weight.compute_sample_weight(class_weight='balanced', y=labels)
class_weights = np.unique(sample_weights)

# Create a dictionary mapping class index to class weight
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

print(class_weights_dict)

samples_per_class = sample_per_class_counter(y_train)
weights_per_class = torch.tensor([total_samples / class_sample_size for class_sample_size in samples_per_class], dtype=torch.float32)
weights_per_class

rescaling_factor = total_samples / torch.sum(weights_per_class)
weights_per_class *= rescaling_factor
weights_per_class_dict = {0: weights_per_class[0].item(), 1: weights_per_class[1].item()}
weights_per_class_dict

{0: 0.5177258971033291, 1: 14.603658536585366}


{0: 655.9999389648438, 1: 18504.0}

## Sanity Check for Distribution of Data

In [ ]:
# Find the sample sizes of each class (test)
samples_per_class_test = sample_per_class_counter(y_test)

samples_per_class_test = [100*x/y_test.shape[0] for x in samples_per_class_test]
samples_per_class_test

# Find the sample sizes of each class (train)
samples_per_class_train = sample_per_class_counter(y_train)

samples_per_class_train = [100*x/y_train.shape[0] for x in samples_per_class_train]

# Find the sample sizes of each class (validation)
samples_per_class_val = sample_per_class_counter(y_val)

samples_per_class_val = [100*x/y_val.shape[0] for x in samples_per_class_val]

print('Train distribution = ', samples_per_class_train)
print('Validation distribution = ', samples_per_class_val)
print('Test distribution = ', samples_per_class_test)

Train distribution =  [96.57620041753654, 3.4237995824634657]
Validation distribution =  [94.96809032891507, 5.031909671084929]
Test distribution =  [96.22687439143135, 3.7731256085686464]


## VGG19 : Raw data / No re-weighting / Pre-trained (No fine-tuning)

In [ ]:
model = keras.applications.VGG19(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
)

for layer in model.layers[:-4]:
    layer.trainable = False

#Adding custom Layers 
# Create the input tensor
inputs = keras.Input(shape=x_train.shape[1:])

# Apply normalization using the `Normalization` layer
normalized = keras.layers.Normalization()(inputs)

# Pass the normalized input through the VGG19 base model
x = model(normalized)

x = keras.layers.GlobalAveragePooling2D()(x)
#x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, activation='relu')(x)
#x = keras.layers.Dense(512, activation='relu')(x)
# x = keras.layers.Dropout(0.2)(x)
#x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
output = keras.layers.Dense(1, activation="sigmoid", name="output")(x)

model_final = keras.Model(inputs =inputs, outputs = output)

model_final.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy',  metrics=[tf.keras.metrics.AUC()])

model_final.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 normalization_2 (Normalizat  (None, 128, 128, 3)      7         
 ion)                                                            
                                                                 
 vgg19 (Functional)          (None, None, None, 512)   20024384  
                                                                 
 global_average_pooling2d_2   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 64)                32832     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0   

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', # Metric to monitor
    patience=20,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True
    )  # Restore the weights of the best epoch

history = model_final.fit(x_train, y_train, 
                    batch_size=32, 
                    epochs=100,  # Increase the number of epochs to allow for early stopping
                    validation_split=0.2,
                    callbacks=early_stopping,
                    class_weight=class_weights_dict) 

Epoch 1/100
479/479 [==============================] - 17s 15ms/step - loss: 5.0180 - auc_2: 0.6763 - val_loss: 1.1398 - val_auc_2: 0.6790
Epoch 2/100
479/479 [==============================] - 6s 12ms/step - loss: 1.0388 - auc_2: 0.7957 - val_loss: 0.6339 - val_auc_2: 0.6849
Epoch 3/100
479/479 [==============================] - 6s 12ms/step - loss: 0.5656 - auc_2: 0.8691 - val_loss: 0.6043 - val_auc_2: 0.7322
Epoch 4/100
479/479 [==============================] - 6s 12ms/step - loss: 0.4502 - auc_2: 0.9005 - val_loss: 0.4665 - val_auc_2: 0.7309
Epoch 5/100
479/479 [==============================] - 6s 12ms/step - loss: 0.3499 - auc_2: 0.9291 - val_loss: 0.4624 - val_auc_2: 0.7180
Epoch 6/100
479/479 [==============================] - 6s 12ms/step - loss: 0.2835 - auc_2: 0.9491 - val_loss: 0.3863 - val_auc_2: 0.7447
Epoch 7/100
479/479 [==============================] - 6s 12ms/step - loss: 0.2627 - auc_2: 0.9584 - val_loss: 0.5345 - val_auc_2: 0.7560
Epoch 8/100
479/479 [============

## Output Metrics for Training / Validation / Test

### Raw | Training

In [ ]:
y_pred_probs = model_final.predict(x_train)

def evaluate_thresholds(y_true, y_pred_probs, thresholds):
  results = []
  for threshold in thresholds:

      # Convert probabilities to binary predictions based on threshold
      y_pred = np.where(y_pred_probs >= threshold, 1, 0)
      
      # Calculate evaluation metrics
      positive_count = sum(y_pred)
      accuracy = accuracy_score(y_true, y_pred)
      precision = precision_score(y_true, y_pred)
      recall = recall_score(y_true, y_pred)
      f1 = f1_score(y_true, y_pred)
      auc = roc_auc_score(y_true, y_pred)
      
      results.append((threshold, positive_count, f1, precision, recall, auc, accuracy))
  
  results_df = pd.DataFrame(results, columns=["Threshold","positive_count" ,"F1", "Precision", "Recall", "AUC", "Accuracy"])
  return results_df

results_df = evaluate_thresholds(y_train, y_pred_probs, np.arange(0, 1, 0.05))
results_df

599/599 [==============================] - 6s 9ms/step


,Threshold,positive_count,F1,Precision,Recall,AUC,Accuracy
0,0.00,[19160],0.066209,0.034238,1.000000,0.500000,0.034238
1,0.05,[3938],0.264258,0.154139,0.925305,0.872645,0.823591
2,0.10,[3374],0.295782,0.176645,0.908537,0.879203,0.851879
3,0.15,[2976],0.324890,0.198253,0.899390,0.885223,0.872025
4,0.20,[2735],0.347390,0.215356,0.897866,0.890945,0.884499
5,0.25,[2528],0.368719,0.232199,0.894817,0.894960,0.895094
6,0.30,[2317],0.390851,0.250755,0.885671,0.895927,0.905480
7,0.35,[2160],0.407670,0.265741,0.875000,0.894644,0.912944
8,0.40,[1982],0.432904,0.288093,0.870427,0.897087,0.921921
9,0.45,[1820],0.456381,0.310440,0.861280,0.896729,0.929749


### Raw | Validation

In [ ]:
y_pred_probs = model_final.predict(x_val)

results_df = evaluate_thresholds(y_val, y_pred_probs, np.arange(0, 1, 0.05))
results_df

128/128 [==============================] - 1s 11ms/step


,Threshold,positive_count,F1,Precision,Recall,AUC,Accuracy
0,0.00,[4074],0.095817,0.050319,1.000000,0.500000,0.050319
1,0.05,[844],0.221163,0.137441,0.565854,0.688846,0.799460
2,0.10,[722],0.209277,0.134349,0.473171,0.655815,0.820079
3,0.15,[635],0.202381,0.133858,0.414634,0.636239,0.835542
4,0.20,[577],0.191816,0.129983,0.365854,0.618052,0.844870
5,0.25,[527],0.196721,0.136622,0.351220,0.616809,0.855670
6,0.30,[483],0.194767,0.138716,0.326829,0.609654,0.864016
7,0.35,[453],0.191489,0.139073,0.307317,0.603258,0.869416
8,0.40,[426],0.190174,0.140845,0.292683,0.599042,0.874570
9,0.45,[398],0.185738,0.140704,0.273171,0.592388,0.879480


### Raw | Test

In [ ]:
y_pred_probs = model_final.predict(x_test)

results_df = evaluate_thresholds(y_test, y_pred_probs, np.arange(0, 1, 0.05))
results_df

129/129 [==============================] - 1s 11ms/step


,Threshold,positive_count,F1,Precision,Recall,AUC,Accuracy
0,0.00,[4108],0.072719,0.037731,1.000000,0.500000,0.037731
1,0.05,[637],0.186869,0.116170,0.477419,0.667498,0.843233
2,0.10,[528],0.190337,0.123106,0.419355,0.651114,0.865385
3,0.15,[461],0.188312,0.125813,0.374194,0.636123,0.878286
4,0.20,[419],0.184669,0.126492,0.341935,0.624674,0.886076
5,0.25,[379],0.176030,0.124011,0.303226,0.609619,0.892892
6,0.30,[342],0.173038,0.125731,0.277419,0.600890,0.899951
7,0.35,[306],0.169197,0.127451,0.251613,0.592035,0.906767
8,0.40,[280],0.174713,0.135714,0.245161,0.591971,0.912610
9,0.45,[260],0.183133,0.146154,0.245161,0.594501,0.917478


## Map Raw to Patient

In [ ]:
def output_mapper(model, dataset, idx, x, chosen_threshold, y_true):
  
  keys = list(idx.keys())

  # Indexing referenced relative to the FULL dataset (i.e., index 0 to 27k)
  if dataset == 'train':
    start_key_pos = 0
    end_key_pos = np.int(0.7*len(keys))
    sliced_keys = keys[start_key_pos : end_key_pos+1]
    
  elif dataset == 'val':
    start_key_pos = np.int(0.7*len(keys))+1
    end_key_pos = np.int(0.85*len(keys))
    sliced_keys = keys[start_key_pos : end_key_pos+1]
    # print(sliced_keys)

  elif dataset == 'test':
    start_key_pos = np.int(0.85*len(keys))+1
    sliced_keys = keys[start_key_pos : len(keys)]

  y_pred_probs = model.predict(x)
  y_pred = np.where(y_pred_probs >= chosen_threshold, 1, 0)
  true_y_mapped = []
  pred_y_mapped = []

  for i in range(0, len(sliced_keys)):

    # Extract indexing for start and end image per patient
    start_img_idx = idx[sliced_keys[i]][0]    # 0
    end_img_idx = idx[sliced_keys[i]][1]      # 31
    # print(start_img_idx, end_img_idx)

    # Slice y true and predicted
    sliced_y = y_true[start_img_idx : end_img_idx]

    if dataset == 'val' or dataset == 'test':
      start_img_idx = start_img_idx - idx[sliced_keys[0]][0]
      end_img_idx = end_img_idx - idx[sliced_keys[0]][0]
      # print(start_img_idx, end_img_idx)
    
    sliced_y_pred = y_pred[start_img_idx : end_img_idx]

    # Store in lists
    if 1 in sliced_y:
      true_y_mapped.append(1)
    else:
      true_y_mapped.append(0)
    
    if 1 in sliced_y_pred:
      pred_y_mapped.append(1)
    else:
      pred_y_mapped.append(0)  

  return true_y_mapped, pred_y_mapped  

In [ ]:
true_y_train_mapped, pred_y_train_mapped = output_mapper(model_final, 'train', idx, x_train, 0.05, y)
print(len(true_y_train_mapped), len(pred_y_train_mapped))

599/599 [==============================] - 5s 9ms/step
864 864


In [ ]:
true_y_val_mapped, pred_y_val_mapped = output_mapper(model_final, 'val', idx, x_val, 0.05, y)
print(len(true_y_val_mapped), len(pred_y_val_mapped))

128/128 [==============================] - 1s 9ms/step
185 185


In [ ]:
true_y_test_mapped, pred_y_test_mapped = output_mapper(model_final, 'test', idx, x_test, 0.05, y)
print(len(true_y_test_mapped), len(pred_y_test_mapped))

129/129 [==============================] - 1s 9ms/step
184 184


In [ ]:
accuracy_train = accuracy_score(true_y_train_mapped, pred_y_train_mapped)
precision_train = precision_score(true_y_train_mapped, pred_y_train_mapped)
recall_train = recall_score(true_y_train_mapped, pred_y_train_mapped)
f1_train = f1_score(true_y_train_mapped, pred_y_train_mapped)
auc_train = roc_auc_score(true_y_train_mapped, pred_y_train_mapped)

print("----- TRAIN -----")
print(f"F1 Score: {f1_train:.4f}")
print(f"Precision: {precision_train:.4f}")
print(f"Recall: {recall_train:.4f}")
print(f"AUC: {auc_train:.4f}")
print(f"Accuracy: {accuracy_train:.4f}")
print("\n")

accuracy_val = accuracy_score(true_y_val_mapped, pred_y_val_mapped)
precision_val = precision_score(true_y_val_mapped, pred_y_val_mapped)
recall_val = recall_score(true_y_val_mapped, pred_y_val_mapped)
f1_val = f1_score(true_y_val_mapped, pred_y_val_mapped)
auc_val = roc_auc_score(true_y_val_mapped, pred_y_val_mapped)

print("----- VALIDATION -----")
print(f"F1 Score: {f1_val:.4f}")
print(f"Precision: {precision_val:.4f}")
print(f"Recall: {recall_val:.4f}")
print(f"AUC: {auc_val:.4f}")
print(f"Accuracy: {accuracy_val:.4f}")
print("\n")

accuracy_test = accuracy_score(true_y_test_mapped, pred_y_test_mapped)
precision_test = precision_score(true_y_test_mapped, pred_y_test_mapped)
recall_test = recall_score(true_y_test_mapped, pred_y_test_mapped)
f1_test = f1_score(true_y_test_mapped, pred_y_test_mapped)
auc_test = roc_auc_score(true_y_test_mapped, pred_y_test_mapped)

print("----- TEST -----")
print(f"F1 Score: {f1_test:.4f}")
print(f"Precision: {precision_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"AUC: {auc_test:.4f}")
print(f"Accuracy: {accuracy_test:.4f}")
print("\n")

# Define the metrics and corresponding values
metrics = ["F1 Score", "Precision", "Recall", "AUC", "Accuracy"]
train_scores = [f1_train, precision_train, recall_train, auc_train, accuracy_train]
val_scores = [f1_val, precision_val, recall_val, auc_val, accuracy_val]
test_scores = [f1_test, precision_test, recall_test, auc_test, accuracy_test]

# Create the dataframe
data = {"Train": train_scores, "Validation": val_scores, "Test": test_scores}
df = pd.DataFrame(data, index=metrics)

# Print the dataframe
df.transpose()

----- TRAIN -----
F1 Score: 0.3811
Precision: 0.2370
Recall: 0.9732
AUC: 0.7532
Accuracy: 0.5903


----- VALIDATION -----
F1 Score: 0.4539
Precision: 0.2991
Recall: 0.9412
AUC: 0.7222
Accuracy: 0.5838


----- TEST -----
F1 Score: 0.3898
Precision: 0.2614
Recall: 0.7667
AUC: 0.6723
Accuracy: 0.6087




,F1 Score,Precision,Recall,AUC,Accuracy
Train,0.381119,0.236957,0.973214,0.753229,0.590278
Validation,0.453901,0.299065,0.941176,0.722244,0.583784
Test,0.389831,0.261364,0.766667,0.672294,0.608696


## Debugging

In [ ]:
start_key_pos = np.int(0.85*len(keys))+1
end_key_pos = len(keys)
print('Starting key position =', start_key_pos, '| End key position =', end_key_pos)
sliced_keys = keys[start_key_pos : end_key_pos+1]
print('Patient IDs =', sliced_keys)
i = -1
start_img_idx = idx[sliced_keys[i]][0] #- idx[sliced_keys[0]][0]   # 0
end_img_idx = idx[sliced_keys[i]][1] #- idx[sliced_keys[0]][0]    # 31
print('Start image index =', start_img_idx, '| End image index =', end_img_idx)

Starting key position = 1049 | End key position = 1233
Patient IDs = ['11257_1001280', '11259_1001282', '11260_1001283', '11261_1001284', '11262_1001285', '11263_1001286', '11264_1001287', '11265_1001288', '11267_1001290', '11268_1001291', '11269_1001292', '11270_1001293', '11271_1001294', '11272_1001295', '11273_1001296', '11275_1001298', '11276_1001299', '11278_1001301', '11279_1001302', '11280_1001303', '11281_1001304', '11282_1001305', '11284_1001307', '11286_1001309', '11287_1001310', '11288_1001311', '11289_1001312', '11290_1001313', '11291_1001314', '11292_1001315', '11293_1001316', '11294_1001317', '11295_1001318', '11297_1001320', '11298_1001321', '11299_1001322', '11301_1001324', '11303_1001326', '11304_1001327', '11305_1001328', '11306_1001329', '11307_1001330', '11308_1001331', '11309_1001332', '11310_1001333', '11311_1001334', '11312_1001335', '11314_1001337', '11315_1001338', '11316_1001339', '11317_1001340', '11318_1001341', '11319_1001342', '11320_1001343', '11321_10013

In [ ]:
x = 27340  # Value to search for

found_key = None

for key, value in idx.items():
    if value[1] == x:
        found_key = key
        break

print(found_key)

None
